# Задание 28. Криптоанализ шифра простой замены.

## ФАЙЛЫ, НЕОБХОДИМЫЕ ДЛЯ РАБОТЫ БЛОКНОТА

Для корректной работы всего алгоритма необходимо скачать архив с двумя файлами и распаковать в корень проекта. Файлы в проекте:
- `snail_on_the_hill.txt`: текст повести братьев Стругацких "Улитка на склоне", нужен для подсчёта матрицы биграмм открытого текста
- `practice.txt`: текстовый файл со всеми вариантами практики

[Ссылка на архив](https://drive.google.com/file/d/1M-zpeztl2FgXud15oJG9CIBsX9Qxb7fF/view?usp=sharing)

`SHA-1` хэш-сумма архива (ну мало ли, мы же всё-таки криптографией занимаемся): `1420bb1e8ac16335a21332dfc88fefb4d907cc21`

## Подгрузка модулей

In [1]:
! pip install beautifulsoup4 requests pandas

## Идея алгоритма

В данном алгоритме используются матрицы биграмм. Возьмём матрицу биграмм исходного текста $B = (b_{ij})_{n \times n}$ и матрицу биграмм получаемого в процессе дешифрования текста $\Delta(t)=(\Delta_{ij}(t))_{n\times n}$. Зададим некоторую функцию – характеристику дешифрованного текста на основе этих матриц: $$f(D_k(y)) = \sum_{ij} |\Delta_{ij}(D_k(y))-b_{ij}|$$

Ясно, что если текст расшифрован верно, $f(D_k(y)) = 0$

Матрицу биграмм $B$ в данном случае построить невозможно (неизвестен исходный текст), но ясно, что матрица биграмм будет примерно соответствовать матрице биграмм для русского языка (или, как минимум, достаточно крупного текста на русском языке, что и используется ниже). Тогда функцию $f$ можно использовать для оценки "близости" шифротекста к русскому языку. Задача сводится к нахождению такого ключа $k$, что $f(D_k(y))$ минимально.

Ниже приведено решение данной задачи алгоритмом локального поиска. Для начала будет находится некоторый ключ, а затем выбираться по два символа в попытке "улучшить" значение функции $f$.

## Реализация

### Вспомогательные функции

Зададим алфавит – буквы от "а" до "я" (все 33 буквы):

In [2]:
ALPHABET = "абвгдеёжзийклмнопрстуфхцчшщъыьэюя"

Функции обработки текста и дешифрования обработанного текста ключом:

In [3]:
# Оставляет в тексте только символы из алфавита
def process_text(text, alphabet):
    return ''.join([c for c in text.lower() if c in alphabet])

# Дешифрует (очищенный) текст ключом
def decrypt(ciphertext, alphabet, key):
    return ''.join([alphabet[key.index(c)] if c in alphabet else c for c in ciphertext])

Ниже приведены функции создания матриц биграмм для некоторых текстов (как в виде строки, так и по пути к файлу), а также функция перевода абсолютной частоты в относительную:

In [4]:
import numpy as np

# Создаёт матрицу биграмм по (очищенному) открытому тексту в виде строки
def create_bigram_matrix(text: str, alphabet: str) -> np.ndarray:
    bigram_matrix = np.zeros((len(alphabet), len(alphabet)))
    for i in range(len(text) - 1):
        row = alphabet.index(text[i])
        col = alphabet.index(text[i + 1])
        bigram_matrix[row][col] += 1
    return bigram_matrix

# Создаёт матрицу биграмм по (очищенному) открытому тексту из текстового файла
def create_bigram_matrix_from_file(path: str, alphabet: str) -> np.ndarray:
    text = process_text(open(path, 'r').read(), alphabet)
    return create_bigram_matrix(text, alphabet)

# Преобразует значения матрицы от абсолютной частоты к относительную
def normalize(matrix: np.ndarray) -> np.ndarray:
    return np.divide(matrix, matrix.sum())

Непосредственно функция $f$, значение которой нужно будет минимизировать

In [5]:
def f(current_matrix: np.ndarray, target_matrix: np.ndarray) -> float:
    return np.sum(np.abs(current_matrix - target_matrix))

В основном алгоритме должен быть задан какой-то начальный ключ. В данном случае выбран следующий: буква заменяется в соответствии с частотой шифротекста и открытого текста (русского языка). Самые частые буквы в русском языке заменяются самыми частыми буквами шифротекста, и наоборот:

In [6]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# Здесь точно так же загружается частотный словарь русского языка, как и в индиве
# Но есть некоторые дополнения
def get_base_key(ciphertext: str, alphabet: str) -> list:
    # Прочитать GET-запросом и вытянуть табличку
    response = requests.get("http://dict.ruslang.ru/freq.php?act=show&dic=freq_letters&title=%D7%E0%F1%F2%EE%F2%ED%EE%F1%F2%FC%20%E1%F3%EA%E2%20%F0%F3%F1%F1%EA%EE%E3%EE%20%E0%EB%F4%E0%E2%E8%F2%E0")
    soup = BeautifulSoup(response.text, "html.parser").find_all("table")[1]
    rows = soup.find_all('tr')
    list_rows = []

    # Пробег по табличке и преобразование каждой строки в список
    for row in rows:
        row_td = row.find_all('td')[1:]
        str_cells = list(map(str, row_td))
        clean_text = list(map(lambda x: BeautifulSoup(x,"html.parser").get_text().strip(), str_cells))
        list_rows.append(clean_text)

    # Фронтендер сайтя НКРЯ сделал между заголовком и текстом пустую строку >:(
    # Вырезаем её
    list_rows = list_rows[0:1] + list_rows[2:]

    # Перегонка в пандас датафрейм (можно было обойтись без этого, но вытянуть столбец из датафрейма проще)
    df = pd.DataFrame(list_rows)
    headers = df.iloc[0]
    new_df  = pd.DataFrame(df.values[1:], columns=headers).drop(columns=[None, 'Ранг'])
    new_df = new_df.astype({'Абс. частота': 'int32'}, copy=False)

    # Сортировка по убыванию частоты
    new_df.sort_values(by='Абс. частота', ascending=False, inplace=True)
    # Получается строчка символов в порядке убывания частоты для открытого текста
    target_freq_str = ''.join(list(new_df['Буква']))

    # Подсчёт частоты символов в шифротексте (точно так же, как в индиве)
    freq = [0] * len(alphabet)
    for j in ciphertext:
        freq[alphabet.find(j)] += 1
    # Строчка символов в порядке убывания частоты для шифротекста
    freq = ''.join([x for _, x in sorted(zip(freq, alphabet), reverse=True)])

    # Сборка из трёх строк одного ключа (символ из алфавита -> его позиция в частоте открытого текста ->
    # -> символ на этой позиции в частоте шифротекста)
    base_key = [freq[target_freq_str.index(x)] for x in alphabet]
    return base_key


### Основная часть алгоритма

Тело алгоритма - функция `analyze`. Шаги алгоритма:
1. Создать базовый ключ на основе частот
2. Поменять некоторые два символа в ключе местами
3. Проверить, "улучшилось" ли значение функции $f$
4. Если улучшилось, заменить ключ новым, перейти на шаг два
5. Иначе перейти на шаг 2.

Алгоритм повторяется, пока появляются "улучшения" значения $f$. Если улучшений не наблюдалось уже `when_return` раз, считается, что ключ найден, алгоритм завершает работу.

Для вычисления матрицы биграмм открытого текста используется текст повести братьев Стругацких "Улитка на склоне".

Выбор символов для перемены местами – зацикленный перебор всех $C_n^2$ пар.

In [7]:
# Получает следующую из C_n^2 пару индексов для перестановки
def get_next_combo(i, j, n):
    j += 1
    if j == n:
        i += 1
        j = i + 1
    if i == n - 1:
        i, j = 0, 1
    return i, j

# "Сердце" алгоритма, принимает шифротекст, возвращает ключ
def analyze(ciphertext: str, alphabet: str, when_return: int=1200) -> list:
    # Загрузка матрицы открытого текста (по файлу с текстом "Улитки на склоне")
    target_matrix = normalize(create_bigram_matrix_from_file("snail_on_the_hill.txt", ALPHABET))
    # Изначально лучший ключ задаётся на основе частот
    best_key = get_base_key(ciphertext, alphabet)
    # Подсчёт оценки f для ключа
    best_score = f(normalize(create_bigram_matrix(ciphertext, alphabet)), target_matrix)
    # Счётчик для выхода
    return_counter = 0
    # Изначальная пара i, j
    i, j = 0, 0
    while return_counter < when_return:
        # Выбор пары букв в ключе, перемена их местами
        key = best_key.copy()
        i, j = get_next_combo(i, j, len(key))
        key[i], key[j] = key[j], key[i]
        # Пересчёт оценки для нового ключа
        score = f(normalize(create_bigram_matrix(decrypt(ciphertext, alphabet, key), alphabet)), target_matrix)
        # Стало лучше => надо изменить ключ
        if score < best_score:
            return_counter = 0
            best_key = key
            best_score = score
        return_counter += 1
    return best_key

## Демонстрация

Для демонстрации в случае какого-то одного текста я использовал свой вариант из индива:

In [8]:
import textwrap

input_str = '''звамйзгзажжся сдрцря йзбизжрдс ёжа кзёзт г оадафзжу жргоз жа йзкхзкрд жз эоз аща жр з чаё жа нзизлрдз я ёзндс гукс унзкжз иытор с кзёлсвзожрцы жа выдз кзёс йзэозёу сдрцря йзбизжрдс аща лсб йзбкжз иачалзё р убжсдс зо кзёлсвзожрцы чоз ёажя жао кзёлсвзожрцс жа иркадс ёажя м измгламажья и ёзат гзёжсоа жзлёсдьжыт вамйзлякзг
жс мдакующрт кажь уеа жа жс шуогу звамйзгзажжся сдрцря м уолс ирмадс жс оадафзжа ёажя жрнка жа выдз кзёзт жс жзчь я жа избилсщсдсмь жргоз жрчанз жа бжсд звз ёжа лсммйлзмы сдрцрр зчажь имолаизердр сжроу м гзозлзт я кзнзизлрдсмь имолаороьмя из иозлжрг жз жа йлршдс р жа йзксисдс жргсгрх иамоат с иакь сжрос йалаизкрдс ёзю гжрегу и чаё я выдс бсржоаламзисжс гукс вздьша аа иамь иачал зжс выдс иыжуекажс йалаизкроь зкжс бдрдсмь жсбисжрисдс ёжа с ёажя има жа выдз р жа выдз
има эоз бсмосирдз сдрцрю бскуёсоьмя йзкуёси зжс иачалзё и чаоиалн йзмда лсвзоы йлршдс гз ёжа кзёзт йзнзизлри м кзёлсвзожрцат зжс змёзоладс гислорлу йлзиалрдс жсдрчра ёзрх иащат йлзчдс изйлагр мизрё йлржцрйсё бсйлсидажжза и йршущую ёсшржгу ёза йрмьёз г ёрхсду хзоя эоз ат жрчанз жа ксдз рвз йрмьёз мзмозядз и змжзижзё рб лсммуекажрт жс оаёу гсгзиы шсжмы фдзлажм жс йзваку и зчалакжых мгсчгсх йзозё жсйрдсмь гзфа йзмркадс бс моздзё р жрчанз жа лашрдс гсгзажрвукь дювзижза йлргдючажра жа йзхзеа жс ёажя уе мгзлаа ёзежз йлакйздзероь чоз ёжа зчалакжзт лсб чозоз иоаёяшрдзмь и нздзиу р я лашрдс жаёакдажжз ахсоь и йздьшу йлрчаё ахсоь и чаё выдс  всб иащат ваб кажан ваб кзгуёажози гзозлыа даесдр и мозда р млакр гзозлых жа хисосдз оздьгз йсмйзлос сдрцря звбизжрдс има вздьжрцы бизжрдс и йздрцрю р йзеслжую гзёсжку жргоз звз ёжа жрчанз жа бжсд я гсг мгизбь баёдю йлзисдрдсмь
'''

# Обработка строчки, удаление всего, что не в алфавите
input_str_processed = process_text(input_str, ALPHABET)

# Вычисление ключа с помощью алгоритма, применение его
key = analyze(input_str_processed, ALPHABET)
new_str = decrypt(input_str, ALPHABET, key)
print(f"Ключ: {''.join(key)}")
print(textwrap.fill(new_str, 80))

Ключ: свинкапебртгдёжзйлмоуфхцчшщъыьэюя
обеспокоенная алиция позвонила мне домой к телефону никто не подходил но это еще
ни о чем не говорило я могла куда угодно выйти а домработницы не было дома
поэтому алиция позвонила еще раз поздно вечером и узнала от домработницы что
меня нет домработница не видела меня с воскресенья в моей комнате нормальный
беспорядок на следующий день уже не на шутку обеспокоенная алиция с утра висела
на телефоне меня нигде не было домой на ночь я не возвращалась никто ничего не
знал обо мне расспросы алиции очень встревожили аниту с которой я договорилась
встретиться во вторник но не пришла и не подавала никаких вестей а ведь анита
переводила мою книжку в чем я была заинтересована куда больше ее весь вечер она
была вынуждена переводить одна злилась названивала мне а меня все не было и не
было все это заставило алицию задуматься подумав она вечером в четверг после
работы пришла ко мне домой поговорив с домработницей она осмотрела квартиру
проверила наличие моих

А теперь можно и повеселиться: давайте сделаем за всех индив и посчитаем все варианты в автоматизированном виде :)

In [9]:
import re
import textwrap

with open('output.txt', 'w') as out:
    # Этими двумя строчками парсится текстовый файл с вариантами
    # Используется как обработанный (очищенный) вариант текста, так и "сырой",
    # с пробелами и переносами (для вывода в файл)
    raw_ciphertexts = re.split(r"(\d+)", open('practice.txt', 'r').read())
    ciphertexts = filter(lambda t: len(t[0]) > 0, zip(map(lambda t: process_text(t, ALPHABET), raw_ciphertexts), raw_ciphertexts))
    for i, (ciphertext, raw_ciphertext) in enumerate(ciphertexts):
        # Расшифровка текста
        key = analyze(ciphertext, ALPHABET)
        decrypted = decrypt(raw_ciphertext.strip(), ALPHABET, key)
        # Запись того, что получилось, в файл
        out.write(str(i + 1) + '\n')
        out.write(ALPHABET + '\n')
        out.write(''.join(key) + '\n')
        out.write(textwrap.fill(decrypted, 80) + '\n')

## Выводы

Реализация алгоритма довольно простая. Написать данный алгоритм и запустить – куда быстрее, чем подбирать вручную. Особенно это актуально для крупных текстов.

Есть, правда, и проблемы. Вот некоторые из них:
- Алгоритм не всегда работает правильно. Возможно, он застряёт в локальном минимуме функции оценки $f$. А может, это проблема того, что частота биграмм в шифротексте - не самый точный способ оценки верности ключа. Что-то получается верно, но не всё, дальше придётся восстанавливать руками по контексту.
- Алгоритм работает **медленно**. Все 49 вариантов индива программа выполняет за 3-4 минуты – почти оскорбительно долго для такого простого шифра.
- Выбор текста для составления матрицы биграмм, кажется, влияет. Вообще, изначально я пытался использовать ту матрицу, что приводилась в примерах криптоанализа простой замены (нашёл её в электронном виде). Но там **не было буквы "ё"**!!! И всё сломалось! Пришлось брать длинный текст. Но есть подозрение, что при использовании разных текстов как открытых результат будет немного разный. 

Возможные варианты улучшения - не просто перебирать пары, а использовать более интересные алгоритмы локального поиска - имитация отжига, генетический алгоритм. Не уверен, что это имеет смысл, но звучит интересно. Это может помочь избавится от локального минимума. Также, можно попробовать использовать *триграммы* вместо *биграмм*, наверняка это "улучшит" алгоритм.